In [2]:
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy

from scipy.optimize import curve_fit

In [3]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)

In [4]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [5]:
class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()

In [6]:
class Flux_group_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_weight() == self.group:
            self.collision_sum[-1]  += particle.get_weight()

In [5]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [6]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               

In [7]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)

In [8]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     

In [9]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [10]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [11]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]

In [12]:
def get_quarter_cylinder_estimators(radius, epsilon, zenter, boundary):
    
    estimators = []
    
    points_to_divide = [0.5]
    
    
    cylinder_2 = ZCylinder(0, 0, zenter)
    plane_5 = Plane(0, 0, 1, 1000)
    plane_6 = Plane(0, 0, 1, -1000)

    surfaces = [cylinder_2, plane_5, plane_6]
    signs = [-1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))
    
    for point in points_to_divide:
        
        if point < 1:
            cylinder_1 = ZCylinder(0, 0, point * radius + epsilon)        
        else:
            cylinder_1 = ZCylinder(0, 0, point * radius)
            
        cylinder_2 = ZCylinder(0, 0, point * radius - epsilon)
        

        plane_5 = Plane(0, 0, 1, 1000)
        plane_6 = Plane(0, 0, 1, -1000)

        surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
        signs = [-1, +1, -1, +1]

        cylinder = Cell(surfaces, signs)
        estimators.append(Flux_estimator(cylinder))
    

    cylinder_1 = ZCylinder(0, 0, radius)
            
    cylinder_2 = ZCylinder(0, 0, radius - boundary)
        

    plane_5 = Plane(0, 0, 1, 1000)
    plane_6 = Plane(0, 0, 1, -1000)

    surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
    signs = [-1, +1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))    
        
        
    return estimators

In [13]:
## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml

In [14]:
test_number_of_particles = 40
test_number_interations = 50
test_number_inactive = 5

In [15]:
cylinder_estimators = get_quarter_cylinder_estimators(4.279960, 0.01, 0.01, 0.01)
print(len(cylinder_estimators))

3


In [16]:
test_cylinder = ZCylinder(0, 0, 4.279960)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder = Cell(surfaces, signs)

cylinder.set_boundaries_type(boundaries_type)

cylinder.set_box_sizes(-1000, -4.279960, -1000, 1000, 4.2799605, 1000 )
cylinder.set_zero_point(0., 0., 0.)


In [17]:

k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(cylinder,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, cylinder_estimators)

 num1  2
i == 0
i == 1


KeyboardInterrupt: 

In [299]:
difference = (k_effective_exp[-1] - 1.) * 100000
standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  18.162894872220114  with standart deviation [pcm]+- 81.18882240585226


In [300]:
def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  

In [301]:
def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision
    

In [302]:
average_sum_of_collision = get_average_number_of_collision(estimators)

In [303]:
print(average_sum_of_collision)

[3.417026727709129e-05, 0.02550223610688999, 0.00925205973103971]


In [304]:
def divide_estimators_square(average_number_of_collision, radius, epsilon, zenter, boundary):
    average_number_of_collision[0] /= np.pi * zenter * zenter
    
    average_number_of_collision[1] /= np.pi * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                            (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon))
    
    average_number_of_collision[2] /= np.pi * ((radius) * (radius) -
                                            (radius - boundary) * (radius - boundary))
    
    
    
    result_estimators = [ average_number_of_collision[1]/
                         average_number_of_collision[0], average_number_of_collision[2]/ average_number_of_collision[0]]
    
    return result_estimators, average_number_of_collision[0]

In [305]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()

In [306]:
average_sum_of_collision = get_average_number_of_collision(estimators)

In [307]:
normalise_flux, zero_point = divide_estimators_square(average_sum_of_collision, 4.279960, 0.01, 0.01, 0.01)

In [308]:
print(normalise_flux)

[0.8718869564171515, 0.31668537023459575]


In [309]:
flux_std = get_std_flux(estimators)

In [310]:
print(flux_std)

[7.541440702882037e-06, 0.00019083365261229613, 0.00012356395895018617]


In [311]:
test_flux = [0.8093, 0.2926]

In [312]:
for i in range(0, len(test_flux)):
    print(str(((normalise_flux[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))

7.733467986797418 % 19.083365261229613
8.231500421939755 % 12.356395895018617


In [313]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[17]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy

from scipy.optimize import curve_fit


# In[18]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)


# In[19]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[20]:


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()


# In[21]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[22]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               


# In[23]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)


# In[24]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[25]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[26]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[27]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[28]:


def get_quarter_cylinder_estimators(radius):
    
    estimators = []
    
    points_to_divide = [0.5]
    epsilon = 0.2
    
    
    cylinder_2 = ZCylinder(0, 0, epsilon)
    plane_5 = Plane(0, 0, 1, 1000)
    plane_6 = Plane(0, 0, 1, -1000)

    surfaces = [cylinder_2, plane_5, plane_6]
    signs = [-1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))
    
    for point in points_to_divide:
        
        if point < 1:
            cylinder_1 = ZCylinder(0, 0, point * radius + epsilon)        
        else:
            cylinder_1 = ZCylinder(0, 0, point * radius)
            
        cylinder_2 = ZCylinder(0, 0, point * radius - epsilon)
        

        plane_5 = Plane(0, 0, 1, 1000)
        plane_6 = Plane(0, 0, 1, -1000)

        surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
        signs = [-1, +1, -1, +1]

        cylinder = Cell(surfaces, signs)
        estimators.append(Flux_estimator(cylinder))
    

    cylinder_1 = ZCylinder(0, 0, radius)
            
    cylinder_2 = ZCylinder(0, 0, radius - 0.05)
        

    plane_5 = Plane(0, 0, 1, 1000)
    plane_6 = Plane(0, 0, 1, -1000)

    surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
    signs = [-1, +1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))    
        
        
    return estimators


# In[29]:


## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml


# In[33]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 50


# In[34]:


cylinder_estimators = get_quarter_cylinder_estimators(4.279960)
print(len(cylinder_estimators))


# In[35]:


test_cylinder = ZCylinder(0, 0, 4.279960)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder = Cell(surfaces, signs)

cylinder.set_boundaries_type(boundaries_type)

cylinder.set_box_sizes(-1000, -4.279960, -1000, 1000, 4.2799605, 1000 )
cylinder.set_zero_point(0., 0., 0.)


# In[36]:



k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(cylinder,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, cylinder_estimators)





difference = (k_effective_exp[-1] - 1.) * 100000
standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[66]:


def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  


# In[67]:


def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision
    


# In[68]:


average_sum_of_collision = get_average_number_of_collision(estimators)





def divide_estimators_square(average_number_of_collision, radius, epsilon):
    average_number_of_collision[0] /= np.pi * epsilon * epsilon
    
    average_number_of_collision[1] /= np.pi * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                            (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon))
    
    average_number_of_collision[2] /= np.pi * ((radius) * (radius) -
                                            (radius - 0.05) * (radius - 0.05))
    
    
    
    result_estimators = [ average_number_of_collision[1]/
                         average_number_of_collision[0], average_number_of_collision[2]/ average_number_of_collision[0]]
    
    return result_estimators, average_number_of_collision[0]


# In[71]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[72]:


average_sum_of_collision = get_average_number_of_collision(estimators)


# In[73]:


normalise_flux, zero_point = divide_estimators_square(average_sum_of_collision, 4.279960, 0.2)




flux_std = get_std_flux(estimators)



test_flux = [0.8093, 0.2926]


# In[65]:


for i in range(0, len(test_flux)):
    print(str(((normalise_flux[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))


# In[40]:


file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)





3
 num1  8
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 1.0237054697566126   0.9983033045276006  0.025402165229011897
i == 53
 keff_cycle , k_effective_exp, std_k_effective 1.045472337846981   1.0140263156340608  0.02150123432955749
i == 54
 keff_cycle , k_effective_exp, std_k_effective 1.0470826792978918   1.0222904065500185  0.017304529393037137
i == 55
 keff_cycle , k_effective_exp, std_k_effective 1.1014567447018815   1.038123674180391  0.020745129702262555
i == 56
 keff_cycle , k_effective_exp, std_k_effective 1.0024948034239454   1.0321855290543167  0.017949053042267085
i

 keff_cycle , k_effective_exp, std_k_effective 0.9695887896418803   1.0036135556255528  0.0052381642932283565
i == 121
 keff_cycle , k_effective_exp, std_k_effective 1.0318048698729296   1.0040106163896003  0.005179103273586105
i == 122
 keff_cycle , k_effective_exp, std_k_effective 1.039982206409475   1.0045102218065431  0.00513104569091549
i == 123
 keff_cycle , k_effective_exp, std_k_effective 1.02129125843876   1.0047400990206832  0.00506548798118819
i == 124
 keff_cycle , k_effective_exp, std_k_effective 1.0170253902374242   1.004906116469558  0.004999323878354807
i == 125
 keff_cycle , k_effective_exp, std_k_effective 0.9289952768322118   1.00389397194106  0.005034996465490833
i == 126
 keff_cycle , k_effective_exp, std_k_effective 0.9968936821822818   1.0038018628652865  0.004969158573238492
i == 127
 keff_cycle , k_effective_exp, std_k_effective 1.021391537218884   1.004030300194554  0.004909516874796564
i == 128
 keff_cycle , k_effective_exp, std_k_effective 0.9962275797380229

 keff_cycle , k_effective_exp, std_k_effective 0.9474483905425803   1.0026488429023033  0.003411715691133971
i == 192
 keff_cycle , k_effective_exp, std_k_effective 0.9893827492079246   1.0025554197072726  0.0033888922894796864
i == 193
 keff_cycle , k_effective_exp, std_k_effective 1.0322769983457594   1.0027632629145347  0.003371522816006709
i == 194
 keff_cycle , k_effective_exp, std_k_effective 1.0031767369662072   1.0027661342621157  0.003348028828514207
i == 195
 keff_cycle , k_effective_exp, std_k_effective 0.9308416868314713   1.0022701035901802  0.0033616562094383206
i == 196
 keff_cycle , k_effective_exp, std_k_effective 1.0052424951152854   1.0022904624362425  0.0033386138438996522
i == 197
 keff_cycle , k_effective_exp, std_k_effective 0.9721439253747632   1.0020853839528312  0.00332216024087459
i == 198
 keff_cycle , k_effective_exp, std_k_effective 0.932208108889389   1.0016132402024025  0.0033332451357307746
i == 199
 keff_cycle , k_effective_exp, std_k_effective 1.13299

 keff_cycle , k_effective_exp, std_k_effective 0.97472617533408   1.0032607783974747  0.002933006111104041
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.020120774757177   1.0033399333099615  0.0029202766079165685
i == 264
 keff_cycle , k_effective_exp, std_k_effective 0.9725538003705783   1.0031960728756655  0.0029101563896416515
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.9641302461702977   1.0030143713561055  0.0029022825782540032
i == 266
 keff_cycle , k_effective_exp, std_k_effective 0.9505131428985668   1.0027713101132465  0.0028990222369443793
i == 267
 keff_cycle , k_effective_exp, std_k_effective 1.0613240628931222   1.003041138467071  0.0028982197999453275
i == 268
 keff_cycle , k_effective_exp, std_k_effective 0.9765354670399633   1.0029195528183226  0.002887455594921058
i == 269
 keff_cycle , k_effective_exp, std_k_effective 1.023713935484481   1.0030145043373462  0.0028758085760438637
i == 270
 keff_cycle , k_effective_exp, std_k_effective 0.967095

 keff_cycle , k_effective_exp, std_k_effective 0.9562804614981056   1.002443395138725  0.0025219510039405286
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9823204090038175   1.002372289180651  0.002514029485331705
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.0338917437913615   1.0024832731757591  0.002507618837406537
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.008867565810648   1.0025056742025482  0.0024989050913592262
i == 336
 keff_cycle , k_effective_exp, std_k_effective 0.9974073957837508   1.0024878480542307  0.0024902161369275
i == 337
 keff_cycle , k_effective_exp, std_k_effective 0.9128444774091032   1.002175501815049  0.0025011043178689496
i == 338
 keff_cycle , k_effective_exp, std_k_effective 1.0137346719353364   1.0022156378224112  0.0024927279388919626
i == 339
 keff_cycle , k_effective_exp, std_k_effective 0.9794509512016386   1.0021368672804707  0.002485336203191643
i == 340
 keff_cycle , k_effective_exp, std_k_effective 1.030367880

 keff_cycle , k_effective_exp, std_k_effective 1.024182214534558   1.0013303964668485  0.002311265083656092
i == 404
 keff_cycle , k_effective_exp, std_k_effective 1.0787076318833182   1.0015489762279122  0.002315068662346238
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.003419924535607   1.0015542465048353  0.0023085441471293166
i == 406
 keff_cycle , k_effective_exp, std_k_effective 1.051381253235838   1.0016942100068886  0.0023063012695873296
i == 407
 keff_cycle , k_effective_exp, std_k_effective 1.0420769746218286   1.0018073269946053  0.0023026121116174
i == 408
 keff_cycle , k_effective_exp, std_k_effective 1.0268546656101847   1.0018772916276097  0.0022972368943589886
i == 409
 keff_cycle , k_effective_exp, std_k_effective 1.0451851152993699   1.0019979262339378  0.002294003067089916
i == 410
 keff_cycle , k_effective_exp, std_k_effective 1.0205541195177914   1.0020494712152819  0.002288202596621937
i == 411
 keff_cycle , k_effective_exp, std_k_effective 0.942733650

 keff_cycle , k_effective_exp, std_k_effective 0.9503305641345152   1.0022454313914237  0.0021195567348426267
i == 474
 keff_cycle , k_effective_exp, std_k_effective 0.9109484163256834   1.002030108242684  0.0021254866913195114
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.9361915409365925   1.0018751939666695  0.0021261308456791512
i == 476
 keff_cycle , k_effective_exp, std_k_effective 0.9646883764193998   1.0017879009677324  0.002122929521537364
i == 477
 keff_cycle , k_effective_exp, std_k_effective 1.019254613698327   1.001828806618155  0.002118346938724077
i == 478
 keff_cycle , k_effective_exp, std_k_effective 0.9955308736433215   1.001814091821485  0.0021134429607530744
i == 479
 keff_cycle , k_effective_exp, std_k_effective 1.0253414100615885   1.0018689340551448  0.002109223866044662
i == 480
 keff_cycle , k_effective_exp, std_k_effective 1.0152947302512982   1.0019001568369963  0.002104544598819841
i == 481
 keff_cycle , k_effective_exp, std_k_effective 0.9834789